# Data Cleaning

In [2]:
import pandas as pd
from datetime import datetime
import numpy as np

In [4]:
train = pd.read_csv('./datasets/train_users_2.csv')
test = pd.read_csv('./datasets/test_users.csv')

In [6]:
test.shape

(62096, 15)

#### Date account created - to datetime object

In [234]:
train['date_account_created'] = pd.to_datetime(train['date_account_created'])
train['date_first_booking'] = pd.to_datetime(train['date_first_booking'])

test['date_account_created'] = pd.to_datetime(test['date_account_created'])
test['date_first_booking'] = pd.to_datetime(test['date_first_booking'])

In [235]:
print(train[train['date_first_booking'].isna()]['date_account_created'].mean())
print(train[train['date_first_booking'].notna()]['date_account_created'].mean())

2013-07-20 08:46:06.389037568
2013-05-21 09:02:26.254550784


#### Drop rows where first booking occurs before account creation

In [236]:
train.drop(train[train['date_first_booking'] < train['date_account_created']].index, inplace=True)

test.drop(test[test['date_first_booking'] < test['date_account_created']].index, inplace=True)


#### Date first booking to boolean

In [237]:
train['date_first_booking'].fillna(0, inplace=True)
test['date_first_booking'].fillna(0, inplace=True)

In [238]:
train['date_first_booking'] = train['date_first_booking'].map(lambda x: 0 if x==0 else 1);
test['date_first_booking'] = test['date_first_booking'].map(lambda x: 0 if x==0 else 1);


#### Timestamp first active to datetime object

In [239]:
train['timestamp_first_active'] = train['timestamp_first_active'].astype(str).map(lambda x: datetime.strptime(x, '%Y%m%d%H%M%S'))
test['timestamp_first_active'] = test['timestamp_first_active'].astype(str).map(lambda x: datetime.strptime(x, '%Y%m%d%H%M%S'))


In [240]:
np.set_printoptions(precision=None, suppress=True)

In [241]:
#getting the true age for users who entered their birth year
def clean_age(cell):
    if cell > 1000:
        return 2014 - cell
    else:
        return cell
train['age'] = train['age'].map(clean_age)
test['age'] = test['age'].map(clean_age)
#imputing mean age for missing ages
mean_age = train[(train['age'] <= 100) & (train['age'] >= 18)]['age'].mean()
train['age'].fillna(mean_age, inplace=True)
test['age'].fillna(mean_age, inplace=True)
#dropping rows where age is less than 16
train = train[train['age'] >= 16].copy()
#imputing mean age for test data where age is < 16
def less_than_16(cell):
    if cell <=16:
        return mean_age
    else: 
        return cell
test['age'] = test['age'].map(less_than_16)
#dropping ages that are greater than 100
train = train[train['age'] <= 100].copy()
#imputing test ages with mean age
def greater_than_100(cell):
    if cell >=100:
        return mean_age
    else: 
        return cell
test['age'] = test['age'].map(greater_than_100)

In [242]:
sessions_df = pd.read_csv('./sessions.csv')
sessions_df.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [243]:
time_spent = sessions_df.groupby('user_id')['secs_elapsed'].sum()
time_spent.head()

user_id
00023iyk9l     867896.0
0010k6l0om     586543.0
001wyh0pz8     282965.0
0028jgx1x1     297010.0
002qnbzfs5    6487080.0
Name: secs_elapsed, dtype: float64

In [244]:
train = train.merge(time_spent, how='left', left_on='id', right_on=time_spent.index)
test = test.merge(time_spent, how='left', left_on='id', right_on=time_spent.index)


In [245]:
train['lang_dist'] = train['language'].map({'de': 72.61, 'en': 0.0, 'fr': 92.06, 'it': 89.4, 'nl': 63.22, 'pt': 95.45, 'es': 92.25})
test['lang_dist'] = test['language'].map({'de': 72.61, 'en': 0.0, 'fr': 92.06, 'it': 89.4, 'nl': 63.22, 'pt': 95.45, 'es': 92.25})


In [246]:
train['lang_dist'].fillna(0, inplace=True)
test['lang_dist'].fillna(0, inplace=True)


In [247]:
mean_secs_elapsed = train[train['secs_elapsed'].notnull()]['secs_elapsed'].mean()
train['secs_elapsed'].fillna(mean_secs_elapsed, inplace=True)
test['secs_elapsed'].fillna(mean_secs_elapsed, inplace=True)


In [253]:
train['first_affiliate_tracked'].fillna('None', inplace=True)
test['first_affiliate_tracked'].fillna('None', inplace=True)

In [254]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 211041 entries, 0 to 211040
Data columns (total 18 columns):
id                         211041 non-null object
date_account_created       211041 non-null datetime64[ns]
timestamp_first_active     211041 non-null datetime64[ns]
date_first_booking         211041 non-null int64
gender                     211041 non-null object
age                        211041 non-null float64
signup_method              211041 non-null object
signup_flow                211041 non-null int64
language                   211041 non-null object
affiliate_channel          211041 non-null object
affiliate_provider         211041 non-null object
first_affiliate_tracked    211041 non-null object
signup_app                 211041 non-null object
first_device_type          211041 non-null object
first_browser              211041 non-null object
country_destination        211041 non-null object
secs_elapsed               211041 non-null float64
lang_dist              

In [255]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62096 entries, 0 to 62095
Data columns (total 17 columns):
id                         62096 non-null object
date_account_created       62096 non-null datetime64[ns]
timestamp_first_active     62096 non-null datetime64[ns]
date_first_booking         62096 non-null int64
gender                     62096 non-null object
age                        62096 non-null float64
signup_method              62096 non-null object
signup_flow                62096 non-null int64
language                   62096 non-null object
affiliate_channel          62096 non-null object
affiliate_provider         62096 non-null object
first_affiliate_tracked    62096 non-null object
signup_app                 62096 non-null object
first_device_type          62096 non-null object
first_browser              62096 non-null object
secs_elapsed               62096 non-null float64
lang_dist                  62096 non-null float64
dtypes: datetime64[ns](2), float64(3), in

In [256]:
train.columns

Index(['id', 'date_account_created', 'timestamp_first_active',
       'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow',
       'language', 'affiliate_channel', 'affiliate_provider',
       'first_affiliate_tracked', 'signup_app', 'first_device_type',
       'first_browser', 'country_destination', 'secs_elapsed', 'lang_dist'],
      dtype='object')

In [262]:
temp_df = pd.concat([train, test], ignore_index=True)

to_dummy = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider',
            'first_affiliate_tracked', 'signup_app', 'first_device_type',
            'first_browser']

temp_df = pd.get_dummies(temp_df, columns=to_dummy, drop_first=True)

train = temp_df.loc[temp_df['country_destination'].notna()]
test = temp_df.loc[temp_df['country_destination'].isna()]

In [266]:
train.head()

,age,country_destination,date_account_created,date_first_booking,id,lang_dist,secs_elapsed,timestamp_first_active,gender_FEMALE,gender_MALE,...,first_browser_Silk,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_UC Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser
0,36.583307,NDF,2010-06-28,0,gxn3p5htnn,0.0,1.512012e+06,2009-03-19 04:32:55,0,0,...,0,0,0,0,0,0,0,0,0,0
1,38.000000,NDF,2011-05-25,0,820tgsjxq7,0.0,1.512012e+06,2009-05-23 17:48:09,0,1,...,0,0,0,0,0,0,0,0,0,0
2,42.000000,other,2011-12-05,1,bjjt8pjhuk,0.0,1.512012e+06,2009-10-31 06:01:29,1,0,...,0,0,0,0,0,0,0,0,0,0
3,36.583307,US,2010-01-01,1,osr2jwljor,0.0,1.512012e+06,2010-01-01 21:56:19,0,0,...,0,0,0,0,0,0,0,0,0,0
4,46.000000,US,2010-01-02,1,lsw9q7uk0j,0.0,1.512012e+06,2010-01-02 01:25:58,1,0,...,0,0,0,0,0,0,0,0,0,0


In [267]:
train.to_csv('./train_01.csv', index=False)
test.to_csv('./test_01.csv', index=False)

In [270]:
train.shape

(211041, 152)

In [271]:
test.shape

(62096, 152)